# LSTM Multiphase Model Training

Note: Make sure to run the notebook in virtualenv. 

## Loading the Data
The code below loads data and labels from `/research/rih-cs/datasets/elvo-multiphase`.

Each phase data is stored under `/research/rih-cs/datasets/elvo-multiphase/preprocessed`.

In [1]:
import os
import pathlib
import typing

import numpy as np

In [18]:
import  logging

def configure_logger():
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.INFO)
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        fmt='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    root_logger.addHandler(handler)

## Train / Test / Val Split
We will iterate through the pos and neg directory of phase1 to get the index of our train/test/val set. 


In [2]:
TRAIN_DATA = ['P25', 'P48', 'P62', 'P72', 'P144', 'P149', 'P1', 'P4', 'P16', 'P21', 'P32', \
'P36', 'P38', 'P52', 'P59', 'P88', 'P89', 'P118', 'P164', 'P232', 'P255', 'P266', 'P280', \
'P289', 'P73', 'P78', 'P120', 'P142', 'P126', 'P145', 'P147', 'P3', 'P5', 'P6', 'P15', \
'P17', 'P22', 'P28', 'P29', 'P34', 'P57', 'P58', 'P61', 'P66', 'P68', 'P70', 'P77', 'P80', \
'P85', 'P87', 'P94', 'P102', 'P106', 'P107', 'P110', 'P125', 'P127', 'P130', 'P134', 'P135', \
'P141', 'P150', 'P152', 'P153', 'P158', 'P163', 'P166', 'P179', 'P180', 'P181', 'P182', 'P185', \
'P207', 'P209', 'P210', 'P216', 'P218', 'P222', 'P224', 'P225', 'P231', 'P8', 'P13', 'P18', \
'P24', 'P33', 'P40', 'P43', 'P44', 'P47', 'P51', 'P53', 'P56', 'P63', 'P67', 'P69', 'P81', \
'P100', 'P101', 'P111', 'P117', 'P124', 'P146', 'P168', 'P184', 'P187', 'P188', 'P208', 'P212', \
'P248', 'P112', 'P2', 'P10', 'P20', 'P26', 'P46', 'P60', 'P79', 'P93', 'P95', 'P98', 'P116', 'P121', \
'P136', 'P143', 'P148', 'P160', 'P189', 'P203', 'P71', 'P97', 'P140', 'P84', 'P92', 'P131', 'P7', \
'P42', 'P129', 'P137', 'P154', 'P159', 'P176', 'P201', 'P213', 'P9', 'P11', 'P12', 'P19', 'P23', \
'P27', 'P30', 'P31', 'P35', 'P39', 'P45', 'P54', 'P55', 'P64', 'P65', 'P74', 'P91', 'P96', 'P99', \
'P104', 'P105', 'P108', 'P109', 'P113', 'P114', 'P119', 'P122', 'P123', 'P128', 'P132', 'P133', 'P139', \
'P151', 'P155', 'P156', 'P157', 'P165', 'P169', 'P173', 'P174', 'P177', 'P183', 'P186', 'P190', 'P192', \
'P193', 'P194', 'P197', 'P199', 'P200', 'P202', 'P205', 'P14', 'P41', 'P49', 'P75', 'P83', 'P86', 'P90', \
'P103', 'P167', 'P171', 'P196', 'P198', 'P204', 'P214', 'P254', 'P191'] 

TEST_DATA = ['P252', 'P265', 'P162', 'P170', 'P172', 'P178', 'P195', 'P221', 'P253', 'P234', 'P236', 'P237', \
'P241', 'P262', 'P272', 'P277', 'P282', 'P284', 'P285', 'P288', 'P291', 'P293', 'P296', 'P220', 'P228', \
'P246', 'P250', 'P270', 'P273', 'P283', 'P302', 'P268', 'P292', 'P226', 'P245', 'P263', 'P269', 'P286', \
'P217', 'P219', 'P233', 'P244', 'P206', 'P211', 'P215', 'P223', 'P227', 'P235', 'P243', 'P257', 'P258', \
'P260', 'P261', 'P267', 'P275', 'P278', 'P264', 'P274', 'P276', 'P279', 'P242']

VAL_DATA = ['P271', 'P259', 'P238', 'P281', 'P229', 'P240', 'P297', 'P309', 'P310', 'P50', 'P76', 'P230', \
'P304', 'P305', 'P306', 'P307', 'P308', 'P300', 'P290', 'P298', 'P299', 'P249', 'P239', 'P294', 'P301', \
'P303', 'P161', 'P256', 'P37', 'P287', 'P295', 'P82', 'P247'] 

In [3]:
data_path = '/research/rih-cs/datasets/elvo-multiphase/preprocessed/'

In [24]:
LENGTH, WIDTH, HEIGHT = (3, 230, 230)
TRAIN_INDICES = []
TEST_INDICES = []
VAL_INDICES = []

# Usage: np.stack(train_arrays)
train_arrays = []
test_arrays = []
val_arrays = []

def load_training_data(): 
    """
    Returns 4D matrix of training data
    Data is in the form (n_samples, 1, w, h). 
    Samples are sorted by the file name specified in TRAIN_DATA
    """

    phase1_pos_files = sorted(os.listdir(data_path + 'phase1/pos/'))
    for i, filename in enumerate(phase1_pos_files):
        arr = np.load(data_path + 'phase1/pos/' + filename)
        if arr.shape == (LENGTH, WIDTH, HEIGHT):
            matching_name = os.path.splitext(filename)[0] 
            if matching_name in TRAIN_DATA:
                train_arrays.append(arr)
                TRAIN_INDICES.append(i)
            elif matching_name in TEST_DATA: 
                test_arrays.append(arr)
                TEST_INDICES.append(i)
            elif matching_name in VAL_DATA: 
                val_arrays.append(arr)
                VAL_INDICES.append(i)
            else: 
                logging.info(
                f'training file {filename}, {matching_name} is not found.')
        else:
            logging.info(
                f'training file {filename} has incorrect shape {arr.shape}')

In [25]:
load_training_data()

In [26]:
# check the indices 
TRAIN_INDICES, TEST_INDICES, VAL_INDICES

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  27,
  29,
  32,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  46,
  47,
  48,
  49,
  51,
  52],
 [26, 30, 31, 45],
 [])

In [27]:
# check the shape of train array output
np.stack(train_arrays).shape

(44, 3, 230, 230)

## Processing into one input for LSTM

In [21]:
# TODO: set up the multiple (three) parallel phases as input for the LSTM model 
# Doc: https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/
# For example, TRAIN_INDICES has train indices for each phase; train_arrays has values in each phase. 
# To merge them into LSTM's input, do the following steps:
# 1. define these data as a matrix of 3 columns (phases) with n rows 
# 2. data = data.reshape(1, n, 3)
# 3. Check the shape by print(data.shape)

## Build LSTM Model

In [29]:
from keras.layers import Input, BatchNormalization, Dense, Flatten, Embedding
from keras.layers.recurrent import RNN, LSTM 
from keras.models import Model, Sequential

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/local/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/local/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for s

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(n, 3)))
model.add(Dense(1))